In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import wandb

In [11]:
def set_seed(seed = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [12]:
wandb.init(project="Simple NN on MNIST pytorch", save_code=True,
           config={
               "batch_size": 64,
               "epochs": 10,
               "optimizer": "Adam",
               "loss function": "CrossEntropyLoss",
           }
           )

config = wandb.config

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using device {device}")

using device cuda:0


In [14]:
#Prepare the MNIST dataset in this cell
''
train_transform = transforms.Compose([transforms.ToTensor(),
                                transforms.RandomRotation(degrees=10),
                                transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
                                transforms.Normalize((0.5,), (0.5,))])
''
test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=test_transform)
print(f"training data shape: {trainset.data.shape}")
print(f"training labels shape: {trainset.targets.shape}")
print(f"testing data shape: {testset.data.shape}")
print(f"testing labels shape: {testset.targets.shape}")
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle = True)

training data shape: torch.Size([60000, 28, 28])
training labels shape: torch.Size([60000])
testing data shape: torch.Size([10000, 28, 28])
testing labels shape: torch.Size([10000])


In [ ]:
class Simple_Neural_network(nn.Module):
    def __init__(self):
        super(Simple_Neural_network, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 16)
        self.fc3 = nn.Linear(16, 10)
        #self.pool = nn.MaxPool2d(2, 2)

    def forward(self,x):
        #print(f"Shape after conv3 and pool: {x.shape}")
        x = self.flatten(x)
        x= F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Simple_Neural_network()
net.to(device)

Simple_Neural_network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=10, bias=True)
)

In [16]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)#does net.parameters just make a vector of all the parameters in the model?

In [17]:
def train(net, device, train_loader, optimizer, epoch):
    net.train()
    # tqdm creates the progress bar
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
    train_correct =0
    train_total=0

    for i, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = net(data)
        loss = loss_function(outputs, target)
        loss.backward()
        optimizer.step()
        
        # Update the bar with the latest loss
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == target).sum().item()
        train_total += len(data)
        train_accuracy = 100.0 * train_correct / train_total
        pbar.set_description(desc=f'Epoch {epoch} Loss={loss.item():.4f} Accuracy={train_accuracy:.2f}%')
        
        # Log training loss and accuracy to wandb
        wandb.log({"train_loss": loss.item(), "train_accuracy": train_accuracy})
        pbar.set_postfix({'loss': loss.item(), 'accuracy': train_accuracy})

def test(net, device, testloader):
    net.eval()
    correct_labels = 0
    test_loss = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            test_loss += loss_function(outputs, labels).item()
            #print(f"outputs.data = {outputs.data}")
            _, predicted = torch.max(outputs, 1)
            #if (predicted == labels).sum().item():
            correct_labels += (predicted == labels).sum().item()
    test_loss /= len(testloader.dataset)
    accuracy = 100.0 * correct_labels / len(testloader.dataset)
    wandb.log({"test_accuracy": accuracy, "test_loss": test_loss})
    print(f"\n Test Set: Accuracy: {accuracy:.4f}%, test_loss: {test_loss} \n")
# Log test results to wandb
    return accuracy

In [18]:
for epoch in range(1, config.epochs + 1):
    train(net, device, trainloader, optimizer, epoch)
    test_accuracy = test(net, device, testloader)
wandb.finish()

Epoch 1 Loss=0.4665 Accuracy=62.24%: 100%|██████████| 938/938 [01:16<00:00, 12.25it/s, loss=0.467, accuracy=62.2]



 Test Set: Accuracy: 91.0000%, test_loss: 0.005188927859812975 



Epoch 2 Loss=0.2140 Accuracy=83.65%: 100%|██████████| 938/938 [02:19<00:00,  6.74it/s, loss=0.214, accuracy=83.6]



 Test Set: Accuracy: 94.4900%, test_loss: 0.0032341648490168155 



Epoch 3 Loss=0.3421 Accuracy=87.91%: 100%|██████████| 938/938 [01:34<00:00,  9.91it/s, loss=0.342, accuracy=87.9]


...

Epoch 9 Loss=0.1733 Accuracy=92.28%: 100%|██████████| 938/938 [02:00<00:00,  7.81it/s, loss=0.173, accuracy=92.3] 



 Test Set: Accuracy: 97.0600%, test_loss: 0.0015328967893961816 



Epoch 10 Loss=0.0426 Accuracy=92.81%: 100%|██████████| 938/938 [02:10<00:00,  7.18it/s, loss=0.0426, accuracy=92.8]



 Test Set: Accuracy: 97.1900%, test_loss: 0.001455710602621548 



test_accuracy,▁▅▅▆▅▆▇███
test_loss,█▄▃▂▃▂▂▂▁▁
train_accuracy,▁▁▁▂▃▆▆▆▇▇▇▇▇▇▇▇████████████████████████
train_loss,█▇▆▄▄▄▂▃▃▂▃▂▃▂▂▁▂▁▃▂▂▃▃▃▃▂▂▂▂▂▁▂▁▃▂▂▃▁▂▃
test_accuracy,97.19
test_loss,0.00146
train_accuracy,92.815
train_loss,0.04265
